In [3]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# XGBoost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# ✅ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

from prophet.make_holidays import make_holidays_df

train_station = pd.read_csv("/content/drive/Shared drives/Time Series/divvy_data/prod/station/divvy_station_train.csv")
test_station = pd.read_csv("/content/drive/Shared drives/Time Series/divvy_data/prod/station/divvy_station_test.csv")
train_station.head()

Mounted at /content/drive


<ipython-input-3-3778532e6424>:21: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  train_station = pd.read_csv("/content/drive/Shared drives/Time Series/divvy_data/prod/station/divvy_station_train.csv")


,date,start_station_name,start_station_id,total_rides,temp_min_c,rain_sum_mm,snowfall_sum_cm,month,dayofweek,year
0,2020-01-01,63rd St Beach,101,2,-5.5,0.0,0.0,1,2,2020
1,2020-01-01,Orleans St & Hubbard St,636,5,-5.5,0.0,0.0,1,2,2020
2,2020-01-01,Orleans St & Elm St,23,3,-5.5,0.0,0.0,1,2,2020
3,2020-01-01,Orleans St & Chestnut St (NEXT Apts),620,5,-5.5,0.0,0.0,1,2,2020
4,2020-01-01,Ogden Ave & Roosevelt Rd,434,2,-5.5,0.0,0.0,1,2,2020


In [ ]:
def add_lags_for_stations_(rides_stationwide):
  df = rides_stationwide.copy()

  # Convert date column to datetime
  df['date'] = pd.to_datetime(df['date'])

  # Historical Rides

  # shift each station and concat
  stations = df['start_station_name'].unique()
  station_df_list = []
  lags = [1, 7, 30, 90, 365]

  for i in range(len(stations)-2000):
    if i%100 == 0:
      print(f"{i}/{len(stations)}")
    station_df = df[df['start_station_name'] == stations[i]].copy()
    # make sure all stations have complete dates
    complete_dates = pd.date_range(start=station_df['date'].min(), end=station_df['date'].max(), freq='D')
    station_df = station_df.set_index('date').reindex(complete_dates).reset_index()
    station_df
    for lag in lags:
      station_df.loc[:,f'y_lag{lag}'] = station_df['total_rides'].shift(lag)
      station_df_list.append(station_df)

